In [1]:
import sys
import os
sys.path.append("..")
import argparse
from src.data.ingestion2 import collect_data
from src.data.cleaning2 import clean_data
from src.data.select_features import select_features
from src.data.build_features2 import build_features
from src.data.splitting import split_data_by_group
from src.data.balancing import naive_random_oversampling
from src.data.aligning import align_input_to_target
from src.data.utils import batch_save_df_to_csv
from src.data.constants import GROUP_ID_NAME, TECHNIQUE_ID_NAME, LABEL_NAME

ROOT_FOLDER = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
TARGET_PATH = os.path.join(ROOT_FOLDER, 'data/interim')

In [2]:
collect_data ()

--runing src.data.ingestion2


parsing all relationships: 100%|██████████| 16530/16530 [00:00<00:00, 70078.60it/s]


Saving:	 collected_groups_df.csv
Saved:	collected_groups_df.csv
Saving:	 collected_groups_software_df.csv
Saved:	collected_groups_software_df.csv
Saving:	 collected_techniques_df.csv
Saved:	collected_techniques_df.csv
Saving:	 collected_techniques_mitigations_df.csv
Saved:	collected_techniques_mitigations_df.csv
Saving:	 collected_techniques_detections_df.csv
Saved:	collected_techniques_detections_df.csv
Saving:	 collected_techniques_software_df.csv
Saved:	collected_techniques_software_df.csv
Saving:	 collected_labels_df.csv
Saved:	collected_labels_df.csv
Finished: 7 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim


{'groups_df':         ID           name                                        description  \
 47   G0099       APT-C-36  [APT-C-36](https://attack.mitre.org/groups/G00...   
 13   G0006           APT1  [APT1](https://attack.mitre.org/groups/G0006) ...   
 96   G0005          APT12  [APT12](https://attack.mitre.org/groups/G0005)...   
 129  G0023          APT16  [APT16](https://attack.mitre.org/groups/G0023)...   
 130  G0025          APT17  [APT17](https://attack.mitre.org/groups/G0025)...   
 ..     ...            ...                                                ...   
 83   G0044   Winnti Group  [Winnti Group](https://attack.mitre.org/groups...   
 33   G0102  Wizard Spider  [Wizard Spider](https://attack.mitre.org/group...   
 54   G0128      ZIRCONIUM  [ZIRCONIUM](https://attack.mitre.org/groups/G0...   
 119  G0018      admin@338  [admin@338](https://attack.mitre.org/groups/G0...   
 49   G0045       menuPass  [menuPass](https://attack.mitre.org/groups/G00...   
 
             

In [3]:
t_selected_features = ['platforms', 'tactics', 'defenses_bypassed', 'permissions_required', 'mitigation_ID', 'detection_name', 'software_ID']


In [4]:
technique_features, group_features, interaction_matrix = clean_data(save_as_csv = True)
technique_features, group_features = select_features(technique_feature_names= t_selected_features, group_feature_names=['software_ID'])
technique_features, group_features= build_features (
    technique_features_df = technique_features,
    technique_feature_names = t_selected_features,
    group_features_df = group_features,
    group_features_names = ['software_ID']
)

--runing src.data.cleaning2


Saving:	 X_technique_cleaned.csv
Saved:	X_technique_cleaned.csv
Saving:	 X_group_cleaned.csv
Saved:	X_group_cleaned.csv
Saving:	 y_cleaned.csv
Saved:	y_cleaned.csv
Finished: 3 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
List of exported files saved at:  c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim\cleaned.txt
Saving:	 X_group_selected_features.csv
Saved:	X_group_selected_features.csv
Saving:	 X_technique_selected_features.csv
Saved:	X_technique_selected_features.csv
Finished: 2 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
List of exported files saved at:  c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim\selected_features.txt
--runing src.data.build_features2
Saving:	 X_technique_onehot.csv
Saved:	X_technique_onehot.csv
Saving:	 X_group_onehot.csv
Saved:	X_group_onehot.csv
Finished: 2 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
List of exported files saved at:  c:\Users\vuchi\GT24\hieuchivu-gt24-repo

# Test: If all values in a column are equal to 0
If there exists a column in which all values are equal to 0, the one-hot encoding function is not working properly.

In [5]:
zero_columns = technique_features.columns[technique_features.eq(0).all()]
print (zero_columns)

Index([], dtype='object')


In [7]:
save_intermediary_table = True
TRAIN_CV_TEST_RATIO = [.7,.15, .15]
train_y_df, cv_y_df, test_y_df = split_data_by_group (interaction_matrix, ratio= TRAIN_CV_TEST_RATIO, save_as_csv = save_intermediary_table)

train_y_balanced_df = naive_random_oversampling (train_y_df, save_as_csv= save_intermediary_table)


--runing src.data.splitting
Saving:	 train_y_split_by_group.csv
Saved:	train_y_split_by_group.csv
Saving:	 cv_y_split_by_group.csv
Saved:	cv_y_split_by_group.csv
Saving:	 test_y_split_by_group.csv
Saved:	test_y_split_by_group.csv
Finished: 3 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
List of exported files saved at:  c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim\split.txt
--runing src.data.balancing
Saving:	 train_y_naive_oversampled.csv
Saved:	train_y_naive_oversampled.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim


In [8]:
train_X_technique = align_input_to_target ( feature_df= technique_features,
                                                object= 'technique',
                                                target_df= train_y_balanced_df,
                                                from_set = 'train', save_to_csv= save_intermediary_table)
train_X_group = align_input_to_target ( feature_df= group_features,
                                                object= 'group',
                                                target_df= train_y_balanced_df,
                                                from_set = 'train', save_to_csv= save_intermediary_table)

--runing src.data.aligning
Saving:	 train_X_technique_aligned.csv
Saved:	train_X_technique_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
--runing src.data.aligning
Saving:	 train_X_group_aligned.csv
Saved:	train_X_group_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim


In [9]:
cv_X_technique = align_input_to_target (feature_df= technique_features,
                                                object= 'technique',
                                                target_df = cv_y_df,
                                                from_set= 'cv', save_to_csv= save_intermediary_table)
cv_X_group = align_input_to_target (feature_df= group_features,
                                                object= 'group',
                                                target_df = cv_y_df,
                                                from_set = 'cv', save_to_csv=save_intermediary_table)

--runing src.data.aligning
Saving:	 cv_X_technique_aligned.csv
Saved:	cv_X_technique_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
--runing src.data.aligning
Saving:	 cv_X_group_aligned.csv
Saved:	cv_X_group_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim


In [10]:
test_X_technique = align_input_to_target (feature_df= technique_features,
                                                object= 'technique',
                                                target_df=test_y_df,
                                                from_set= 'test', save_to_csv= save_intermediary_table)
test_X_input = align_input_to_target (feature_df= group_features,
                                                object= 'group',
                                                target_df=test_y_df,
                                                from_set= 'test', save_to_csv= save_intermediary_table)

--runing src.data.aligning
Saving:	 test_X_technique_aligned.csv
Saved:	test_X_technique_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
--runing src.data.aligning
Saving:	 test_X_group_aligned.csv
Saved:	test_X_group_aligned.csv
Finished: file saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim


In [11]:
# remove ID columns in target tables after aligning
train_y_balanced_df = train_y_balanced_df[LABEL_NAME]
cv_y_df = cv_y_df[LABEL_NAME]
test_y_df = test_y_df[LABEL_NAME]

In [12]:
dfs = {
    'train_y_balanced':             train_y_balanced_df,
    'train_X_technique_aligned':    train_X_technique,
    'train_X_group_aligned':        train_X_group,
    'cv_y':                         cv_y_df,
    'cv_X_technique_aligned':       cv_X_technique,
    'cv_X_group_aligned':           cv_X_group,
    'test_y':                       test_y_df,
    'test_X_technique_aligned':     test_X_technique,
    'test_X_group_aligned':         test_X_input,
}
batch_save_df_to_csv (file_name_dfs= dfs, target_path=TARGET_PATH, prefix = 'FINAL', output_list_file = 'FINAL')

Saving:	 FINAL_train_y_balanced.csv
Saved:	FINAL_train_y_balanced.csv
Saving:	 FINAL_train_X_technique_aligned.csv
Saved:	FINAL_train_X_technique_aligned.csv
Saving:	 FINAL_train_X_group_aligned.csv
Saved:	FINAL_train_X_group_aligned.csv
Saving:	 FINAL_cv_y.csv
Saved:	FINAL_cv_y.csv
Saving:	 FINAL_cv_X_technique_aligned.csv
Saved:	FINAL_cv_X_technique_aligned.csv
Saving:	 FINAL_cv_X_group_aligned.csv
Saved:	FINAL_cv_X_group_aligned.csv
Saving:	 FINAL_test_y.csv
Saved:	FINAL_test_y.csv
Saving:	 FINAL_test_X_technique_aligned.csv
Saved:	FINAL_test_X_technique_aligned.csv
Saving:	 FINAL_test_X_group_aligned.csv
Saved:	FINAL_test_X_group_aligned.csv
Finished: 9 files saved to c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim
List of exported files saved at:  c:\Users\vuchi\GT24\hieuchivu-gt24-repo2\data/interim\FINAL.txt
